In [2]:
# LangChain 필요 컴포넌트 임포트
from langchain.chat_models import ChatOpenAI  # LLM 모델 (ChatGPT)
from langchain.document_loaders import UnstructuredFileLoader  # 텍스트 파일 로더
from langchain.text_splitter import CharacterTextSplitter  # 문서 분할기
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings  # 텍스트 임베딩 도구
from langchain.vectorstores import FAISS  # 벡터 데이터베이스
from langchain.storage import LocalFileStore  # 로컬 캐시 저장소
from langchain.prompts import ChatPromptTemplate  # 프롬프트 템플릿
from langchain.schema.runnable import RunnablePassthrough  # 체인 구성 유틸리티

# LLM 모델 초기화 (낮은 temperature로 일관된 응답 유도)
llm = ChatOpenAI(
    temperature=0.1,
)

# 임베딩 캐시 저장소 설정
cache_dir = LocalFileStore("./.cache/")

# 문서 분할 설정
# chunk_size: 각 분할 크기, chunk_overlap: 분할 간 중복되는 부분
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 텍스트 파일 로드
loader = UnstructuredFileLoader("./files/document.txt")

# 문서를 설정된 크기로 분할
docs = loader.load_and_split(text_splitter=splitter)

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()

# 임베딩 결과 캐싱 설정 (API 비용 절감)
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 분할된 문서를 벡터화하여 FAISS에 저장
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# 유사도 검색을 위한 리트리버 생성
# what is retriver?
# 한개의 스트링을 받아 질문이나 그와 관련성이 있는 문서를 얻기위한 쿼리 등
# 출력값은 문서의 리스트임
retriver = vectorstore.as_retriever()

# 질문-답변을 위한 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

# 전체 체인 구성:
# 1. 질문에 관련된 문서 검색
# 2. 검색된 문서를 컨텍스트로 프롬프트 생성
# 3. LLM으로 답변 생성
chain = (
    {
        "context": retriver,  # 문서 검색
        "question": RunnablePassthrough(),  # 사용자 질문 전달
    }
    | prompt
    | llm
)

# Victory Mansions에 대한 설명 요청
chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a place where Winston lives. It is described as a dilapidated apartment building with poor living conditions. The building is run-down, with faulty plumbing and electricity. The atmosphere is grim and oppressive, reflecting the overall bleakness of the society in which Winston resides.')